# Basic usecase

## Problem

In this notebook, we provide examples of different solver configurations for the sample Knapsack Problem with three items. The goal is to put chosen items in the knapsack to achieve maximal cost  with total weight not exceeding `max_weight`.  The `items_weights` and `item_values` sections specify the weight and cost of each item, respectively.

```yaml
problem:
  type: knapsack
  max_weight: 2
  items_weights: [1, 1, 1]
  items_values: [2, 2, 1]
```

## Configuration for QAOA
Configuration below shows how to create a QAOA algorithm instance with 5 layers and local gradient descent optimizer (:py:class:`QmlGradientDescent`) with default 'Adam'.

`angles` indicate variational parameters searched by specified optimizer;
`hyper args` refer to the initial weights in the objective function of the Knapsack Problem..

In [1]:
qaqa_config_yaml = """
problem:
  type: knapsack
  max_weight: 2
  items_weights: [1, 1, 1]
  items_values: [2, 2, 1]
solver:
  type: vqa
  pqc:
    type: qaoa
    layers: 5
  optimizer:
    type: qml
  params_inits:
    angles: [[0.5, 0.5, 0.5, 0.5, 0.5], [1, 1, 1, 1, 1]]
    hyper_args: [1, 2.5, 2.5]
"""

## Configuration for D-Wave Advantage

Configuration for a grid search hyperoptimizer. The objective function penalties (`hyper_args`) are searched within specified `bounds` using provided `steps`. The objective function is solved with D-Wave Advantage.

In [2]:
advantage_config_yaml = """
problem:
  type: knapsack
  max_weight: 2
  items_weights: [1, 1, 1]
  items_values: [2, 2, 1]
solver:
  type: advantage
  hyper_optimizer:
    type: grid
    steps: [0.01, 0.01, 0.01]
    bounds: [[1, 10], [1, 10], [1, 10]]
  params_inits:
    weights: [1, 1, 1]
"""

## Advance configuration

QHyper configuration of the QAOA variant (WF-QAOA) with 5 `layers` and  the local gradient descent Adam `optimizer` (qml). `angles` indicate  initial variational parameters optimized by the method. `hyper_args` refer to the initial objective function penalties searched within `hyper_optimizer` `bounds` by the `CEM` method. `processes`, `samples_per_epoch`, and `epochs` are parameters specific to the `CEM` method.

In [3]:
advance_config_yaml = """
problem:
  type: knapsack
  max_weight: 2
  items_weights: [1, 1, 1]
  items_values: [2, 2, 1]
solver:
  type: vqa
  pqc:
    type: wfqaoa
    layers: 5
    backend: default.qubit
  optimizer:
    type: qml
    optimizer: adam
    steps: 200
    stepsize: 0.005
  hyper_optimizer:
    type: cem
    processes: 4
    samples_per_epoch: 1000
    epochs: 10
    bounds: [[1, 10], [1, 10], [1, 10]]
  params_inits:
    angles: [[0.5, 0.5, 0.5, 0.5, 0.5], [1, 1, 1, 1, 1]]
    hyper_args: [1, 2.5, 2.5]
"""

## Solver execution and results evaluation

Importing required libraries and creating a solver instance from the configuration. Here we use the `QAOA` solver, but other configurations could be used as well.

In [4]:
import yaml
from QHyper.solvers import solver_from_config

config = yaml.safe_load(qaqa_config_yaml)

solver = solver_from_config(config)

Next, we run the solver and print the probabilities.

In [5]:
results = solver.solve()
results.probabilities

{'00000': 0.019256615531008367,
 '00001': 0.00499352962189102,
 '00010': 0.06602494739836554,
 '00011': 0.0013503393228611027,
 '00100': 0.00929763961142956,
 '00101': 0.0007737888637532465,
 '00110': 0.015090504145414452,
 '00111': 0.0010554601420100353,
 '01000': 0.01907665783208572,
 '01001': 0.011946196013858858,
 '01010': 0.016634259547056083,
 '01011': 0.0009502979357583128,
 '01100': 0.005882200822365891,
 '01101': 0.11219790796472451,
 '01110': 0.015331155364085032,
 '01111': 0.0005656815846679773,
 '10000': 0.019076657832085682,
 '10001': 0.011946196013858872,
 '10010': 0.01663425954705607,
 '10011': 0.0009502979357583092,
 '10100': 0.005882200822365878,
 '10101': 0.11219790796472442,
 '10110': 0.015331155364085032,
 '10111': 0.0005656815846679773,
 '11000': 0.007446059664264837,
 '11001': 0.07094646891953676,
 '11010': 0.015617824600733517,
 '11011': 0.012249402344729498,
 '11100': 0.0007147274620250246,
 '11101': 0.25909813092602146,
 '11110': 0.0025109903464748972,
 '11111'

After obtaining the results, we evaluate the solution by calculating the total cost and weight of the items in the knapsack.

In [6]:
from QHyper.util import (
    weighted_avg_evaluation, sort_solver_results, add_evaluation_to_results)

# Evaluate results with weighted average evaluation
print("Evaluation:")
print(weighted_avg_evaluation(
    results.probabilities, solver.problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sorted results:")
sorted_results = sort_solver_results(
    results.probabilities, limit_results=5)

# Add evaluation to results
results_with_evaluation = add_evaluation_to_results(
    sorted_results, solver.problem.get_score)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")

Evaluation:
-1.1747833822414966
Sorted results:
Result: 11101, Prob: 0.2591, Evaluation: 0
Result: 11111, Prob: 0.1484, Evaluation: 0
Result: 01101, Prob: 0.1122, Evaluation: -3
Result: 10101, Prob: 0.1122, Evaluation: -3
Result: 11001, Prob: 0.070946, Evaluation: -4
